**Imports**

In [ ]:
import pandas as pd
import numpy as np

**Load Datasets**

In [ ]:
# Load datasets
opw = pd.read_csv("/content/output perworker.csv")
oph = pd.read_csv("/content/output per hour per worker.csv")
iip = pd.read_csv("/content/Table14.6_stat_year_book_2014.csv")
pwt = pd.read_excel("/content/pwt110.xlsx")

In [ ]:
print("OPH columns:")
print(oph.columns)

print("\nOPW columns:")
print(opw.columns)


OPH columns:
Index(['ref_area.label', 'source.label', 'indicator.label', 'time',
       'obs_value'],
      dtype='object')

OPW columns:
Index(['ref_area.label', 'source.label', 'indicator.label', 'time',
       'obs_value'],
      dtype='object')


**Clean ILOSTAT productivity data**

In [ ]:
oph = (
    oph[oph["ref_area.label"] == "India"]
    .rename(columns={
        "time": "year",
        "obs_value": "output_per_hour"
    })[["year", "output_per_hour"]]
)

opw = (
    opw[opw["ref_area.label"] == "India"]
    .rename(columns={
        "time": "year",
        "obs_value": "output_per_worker"
    })[["year", "output_per_worker"]]
)


In [ ]:
pwt = pd.read_excel(
    "/content/pwt110.xlsx",
    sheet_name="Data"
)

**Clean Penn World Table (India only)**

In [ ]:
# Keep India only
pwt = pwt[pwt["countrycode"] == "IND"]

# Select required variables
pwt = pwt[["year", "rgdpo", "emp", "rkna", "avh"]]

# Per-worker normalization (DSGE logic)
pwt["gdp_per_worker"] = pwt["rgdpo"] / pwt["emp"]
pwt["capital_per_worker"] = pwt["rkna"] / pwt["emp"]


**Merge macro + productivity**

In [ ]:
macro = (
    pwt
    .merge(opw, on="year", how="left")
    .merge(oph, on="year", how="left")
)

**Industrial output (reshape to long)**

In [ ]:
iip_long = iip.melt(
    id_vars=["Industry code", "Description"],
    var_name="year",
    value_name="sector_output"
)

iip_long["year"] = iip_long["year"].str.extract(r"(\d{4})")
iip_long = iip_long.dropna(subset=["year"])
iip_long["year"] = iip_long["year"].astype(int)
iip_long = iip_long.rename(columns={"Description": "sector"})

**Merge everything**

In [ ]:
master = iip_long.merge(macro, on="year", how="left")
master = master.sort_values(["sector", "year"])

**Create growth rates (DSGE dynamics)**

In [ ]:
for col in [
    "sector_output",
    "output_per_worker",
    "output_per_hour",
    "gdp_per_worker"
]:
    master[col + "_growth"] = (
        np.log(master[col]) - np.log(master[col].shift(1))
    )

**Save master dataset**

In [ ]:
import os

# Create the output directory if it doesn't exist
os.makedirs('output', exist_ok=True)

master.to_csv("output/master_dataset.csv", index=False)
print("Saved outputs/master_dataset.csv")

Saved outputs/master_dataset.csv
